3/25 5:45 - currently working to intelligently create the target column

TODO:
push to aws?

make a basic model

In [393]:
for i, date in enumerate(target['date']):
    target_date_index[(date - start_date).days] = i
    

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pickle
import string
import sys
import numpy as np
import time

from src.data_processing import data_processing

In [180]:
# with open('data/predictor.p', 'rb') as file:
#     predictor = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [182]:
fires_df = pd.read_csv('data/Fire_Incidents.csv')

C:\Users\pengj\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (11,12,19,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [176]:
with open('target_date_index.p', 'rb') as file:
    target_date_index = pickle.load(file)

In [1]:
def save(df, target = False):
    path = 'data/' + ('target.p' if target else 'predictor.p')
    with open(path, 'wb') as file:
        pickle.dump(df, file)

In [206]:
def target_pipeline(fires_df):
    """
    filter out non fires
    filter out non-addresses
    filter out single number addresses - these cannot be associated with buildings
    """
    fires_df = fires_df[fires_df['Primary Situation'].apply(lambda x: x[0] == '1')]
    fires_df = fires_df[fires_df['Address'].fillna('').apply(lambda x: len(x) > 0)]
    fires_df = fires_df[['Address', 'Incident Date']]
    fires_df.columns = ['address', 'date']
    fires_df['address'] = fires_df['address'].apply(str.upper)
    fires_df = fires_df.reset_index().drop('index', axis = 1)

In [200]:
fires_df = fires_df[['Address', 'Incident Date']]
fires_df.columns = ['address', 'date']

In [204]:
fires_df['date'] = pd.to_datetime(fires_df['date'])

In [212]:
with open('data/fires_df.p','wb') as file:
    pickle.dump(fires_df, file)

In [236]:
def data_pipeline():
    """
    fix march 2004
    
    convert int dates to datetimes!
    convert string dates to datetimes
    
    lowercase columns, make address column
    filter out "filed" permits
    
    
    """
    data = []
    for i, path in enumerate(os.listdir('data')):
        #I downloaded all the data into files with filename length 12/11. This'll filter out the other stuff.
        if len(path) > 12 or len(path) < 11:
            continue
        try:
            df = pd.read_excel('data/' + path)
        except:
            print(i)
            break
        df.columns = [str.lower(col) for col in df.columns]
        df.columns = df.columns.map(column_name_map)

        try:
            if len(df.columns) == 34:
                df['description'] = df['description'].astype(str) + df['unnamed: 33'].astype(str)
                df = df.drop(['unnamed: 33'], axis = 1)
            if len(df.columns) == 35:
                df['description'] = df['description'].astype(str) + df['unnamed: 33'].astype(str) + df['unnamed: 34'].astype(str)
                df = df.drop(['unnamed: 33', 'unnamed: 34'], axis = 1)
            if len(df.columns) == 42:
                df['description'] = df['description'].astype(str) + df['unnamed: 41'].astype(str)
                df = df.drop(['unnamed: 41'], axis = 1)
            if len(df.columns) == 43:
                df['description'] = df['description'].astype(str) + df['unnamed: 41'].astype(str) + df['unnamed: 42'].astype(str)
                df = df.drop(['unnamed: 41', 'unnamed: 42'], axis = 1)
        except KeyError as e:
            print(path)
            print(e)
            print(i)
            break
        data.append(df) 
        
    #7 is march 2004
    mar_2004_cols = list(data[7].columns)
    mar_2004_cols = mar_2004_cols[:10] + mar_2004_cols[11:]  + [mar_2004_cols[10]]
    #74, 124 have those ints in the date column
    date_cols = ['status_date', 'file_date', 'expiration_date']
    for col in date_cols:
        data[74][col] = data[74][col].apply(int_to_date)
        data[124][col] = data[124][col].apply(int_to_date)
        
    predictor = pd.concat(data)
    
    predictor = predictor[predictor['status'] == 'ISSUED']
    
    for col in date_cols:
        predictor[col] = pd.to_datetime(predictor[col])
    
    predictor['street_number'] = predictor['street_number'].fillna(0).astype(int).astype(str)
    predictor['address'] = predictor[['street_number', 'avs_street_name']].apply(lambda x: ' '.join(x.astype(str)),axis = 1)
    predictor = predictor[~predictor['avs_street_name'].isnull()]
    return predictor

In [40]:
def hasnumber(s):
    return bool(re.search(r'\d', s))

In [238]:
    date_cols = ['status_date', 'file_date', 'expiration_date']

In [266]:
predictor['address'] = predictor[['street_number', 'avs_street_name']].apply(lambda x: ' '.join(x.astype(str)),axis = 1)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [267]:
save(predictor)

In [233]:
date_cols = ['status_date', 'file_date', 'expiration_date']

In [123]:
pred = pred.reset_index().drop('index', axis = 1)

In [145]:
pred['address'] = pred[['street_number', 'avs_street_name']].apply(lambda x: ' '.join(x), axis = 1)

In [1]:
pred

NameError: name 'pred' is not defined

In [130]:
target = predictor[['fires', 'fire_rate']]
target.mean()

fires        0.723892
fire_rate    0.629910
dtype: float64

In [ ]:
def parse_address(string):
    
    bool(re.search(r'\d', inputString))
    return street_number, street_name, 

In [170]:
pd.to_datetime('1/1/1900') + pd.Timedelta('{} days'.format(500))

Timestamp('1901-05-16 00:00:00')

In [751]:
save(predictor)

In [289]:
def int_date_series(series):
    return series.apply(int_to_date)

In [108]:
def int_to_date(date):
    if type(date) == int:
        return pd.to_datetime('12/30/1899') + pd.Timedelta('{} days'.format(date))
    else:
        return date

In [93]:
with open('data/full_data.p', 'wb') as file:
    pickle.dump(data, file)

In [431]:
predictor['expiration_date'] = pd.to_datetime(predictor['expiration_date'])

In [278]:
with open('col_name_map.p', 'wb') as file:
    pickle.dump(column_name_map, file)

In [45]:
with open('col_name_map.p', 'rb') as file:
    column_name_map = pickle.load(file)

In [83]:
#load data and align columns
data = []
for i, path in enumerate(os.listdir('data')):
    #I downloaded all the data into files with filename length 12/11. This'll filter out the other stuff.
    if len(path) > 12 or len(path) < 11:
        continue
    try:
        df = pd.read_excel('data/' + path)
    except:
        print(i)
        break
    df.columns = [str.lower(col) for col in df.columns]
    df.columns = df.columns.map(column_name_map)
    
    try:
        if len(df.columns) == 34:
            df['description'] = df['description'].astype(str) + df['unnamed: 33'].astype(str)
            df = df.drop(['unnamed: 33'], axis = 1)
        if len(df.columns) == 35:
            df['description'] = df['description'].astype(str) + df['unnamed: 33'].astype(str) + df['unnamed: 34'].astype(str)
            df = df.drop(['unnamed: 33', 'unnamed: 34'], axis = 1)
        if len(df.columns) == 42:
            df['description'] = df['description'].astype(str) + df['unnamed: 41'].astype(str)
            df = df.drop(['unnamed: 41'], axis = 1)
        if len(df.columns) == 43:
            df['description'] = df['description'].astype(str) + df['unnamed: 41'].astype(str) + df['unnamed: 42'].astype(str)
            df = df.drop(['unnamed: 41', 'unnamed: 42'], axis = 1)
    except KeyError as e:
        print(path)
        print(e)
        print(i)
        break
    data.append(df) 

178


In [92]:
data[7]['status_date'].iloc[1] 

Timestamp('2004-03-01 00:00:00')

In [285]:
for i, datum in enumerate(data):
    for j, row in datum.iterrows():
        if type(row['status_date']) == int:
            print(i, j, 'has ints')
        if row['status_date']

74 4794
124 4794


In [294]:
predictor = pd.concat(data)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [147]:
pred

,15_day_hold?,application_number,assessor_stories,avs_street_name,avs_street_sfx,block,city,company_name,contact_name,contact_phone,...,street,street_number,street_number.1,street_number_sfx,street_suffix,unit,unit_sfx,voluntary soft-story retrofit,zip_code,address
0,NaN,2.00404e+11,NaN,SHOTWELL,ST,3614,NaN,NaN,NaN,NaN,...,NaN,705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,705 SHOTWELL
1,NaN,2.00403e+11,NaN,DIAMOND,ST,2693,SAN FRANCISCO,WALLACE REMODELING INC,NaN,NaN,...,DAY,142,NaN,NaN,ST,NaN,NaN,NaN,94131,142 DIAMOND
2,NaN,2.00403e+11,NaN,DIAMOND,ST,2693,SAN FRANCISCO,WALLACE REMODELING INC,NaN,NaN,...,DAY,148,NaN,NaN,ST,0.0,NaN,NaN,94131,148 DIAMOND
3,NaN,2.00404e+11,NaN,26TH,ST,6564,SAN FRANCISCO,DERESZYNSKI-DUVAL & ASSOCIATES INC.,NaN,NaN,...,ILLINOIS,4089,NaN,NaN,ST,NaN,NaN,NaN,94124-0000,4089 26TH
4,NaN,2.00311e+11,NaN,47TH,AV,1802,NaN,NaN,NaN,NaN,...,NaN,1343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1343 47TH
5,NaN,2.00403e+11,NaN,VALENCIA,ST,3569,NaN,NaN,NaN,NaN,...,NaN,535,NaN,NaN,NaN,0.0,NaN,NaN,NaN,535 VALENCIA
6,NaN,2.00404e+11,NaN,03RD,ST,3706,OAKLAND,ARROW SIGN CO,NaN,NaN,...,46TH,50,NaN,NaN,AV,NaN,NaN,NaN,94601,50 03RD
7,NaN,2.00404e+11,NaN,47TH,AV,2083,NaN,NaN,PAT BUSCOVICH 235 MONTGOMERY ST SF,NaN,...,NaN,1812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1812 47TH
8,NaN,2.00404e+11,NaN,BALDWIN,CT,4714,SAN FRANCISCO,MARINSHIP CONSTRUCTION,NaN,NaN,...,DORMAN,51,NaN,NaN,AV,NaN,NaN,NaN,94124,51 BALDWIN
9,NaN,2.00305e+11,NaN,WEST CLAY,ST,1336,NaN,NaN,PETER WINKELSTEIN,4.153794e+09,...,NaN,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25 WEST CLAY


In [309]:
save(predictor)

MemoryError: 

In [438]:
l = [1,2,3,4,5]
[i,x for i,x in enumerate(l)]

SyntaxError: invalid syntax (<ipython-input-438-be848b48aba2>, line 2)

In [676]:
data[7]['avs_street_name'];

In [331]:
[i for i,datum in enumerate(data[:72]) if len(datum.columns) > 33]

[]

In [321]:
toy = data[81][~data[81]['unnamed: 42'].isnull()]

In [324]:
len(toy.columns)

43

In [336]:
os.listdir('data')[177]

'predictor.p'

In [29]:
links1 = soup.find_all('a', {'href':re.compile(r'/ftp*')})
base = 'https://sfdbi.org'

'https://sfdbi.org/ftp/uploadedfiles/dbi/reports/monthly_reports/bpfi/March2009FiledIssued_Permits.xls'

In [455]:
predictor = pd.concat(data)


C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [474]:
with open('data/predictor.p', 'wb') as file:
    pickle.dump(predictor,file)

In [470]:
predictor['address'] = predictor[['street_number', 'avs_street_name']].apply(lambda x: ' '.join(x.astype(str)), axis = 1)

In [480]:
target['date'] = pd.to_datetime(target['date'])

In [296]:
save(predictor)

MemoryError: 

In [330]:
del temp

In [320]:
names

['BeautifulSoup',
 'In',
 'Out',
 'a',
 'address',
 'b',
 'column_name_map',
 'd',
 'data',
 'datum',
 'df',
 'end',
 'exit',
 'file',
 'fires',
 'get_ipython',
 'hasnumber',
 'i',
 'int_date_series',
 'int_to_date',
 'j',
 'last',
 'name',
 'np',
 'os',
 'p1',
 'p2',
 'p3',
 'path',
 'pd',
 'pickle',
 'predictor',
 'quit',
 're',
 'real_fires',
 'requests',
 'row',
 's',
 'save',
 'start',
 'string',
 'subset',
 'sys',
 'target',
 'temp',
 'thing',
 'time']

In [673]:
mask = predictor['street_number'].apply(lambda x: isinstance(x, str))

In [119]:
predictor.columns = [str.lower(col) for col in predictor.columns]
predictor['address'] = predictor[['street_number', 'avs_street_name']].apply(lambda x: ' '.join(x.astype(str)),axis = 1)

Format changes:
72, 

In [208]:
def compare(colA, colB, n):
    for i, (a, b) in enumerate(zip(colA, colB)):
        if i >= n:
            break
        if a != b:
            print(i)
            return False
    return True

In [222]:
compare(data[0].columns, data[2].columns, 50)

7


False

In [224]:
data[2].columns, data[0].columns

(Index(['APPLICATION_NUMBER', 'FORM_NUMBER', 'FILE_DATE', 'STATUS_DATE',
        'STATUS', 'STATUS_CODE', 'EXPIRATION_DATE', 'ESTIMATEDCOST',
        'REVISEDCOST', 'EXISTINGUSE', 'EXISTINGUNITS', 'PROPOSEDUSE',
        'PROPOSEDUNITS', 'BLOCK', 'LOT', 'STREET_NUMBER', 'STREET_NUMBER_SFX',
        'AVS_STREET_NAME', 'AVS_STREET_SFX', 'UNIT', 'UNIT_SFX', 'FIRST_NAME',
        'LAST_NAME', 'CONTRACTORPHONE', 'COMPANY_NAME', 'STREET',
        'STREET_SUFFIX', 'CITY', 'STATE', 'ZIP_CODE', 'CONTACT_NAME',
        'CONTACT_PHONE', 'DESCRIPTION', 'Unnamed: 33'],
       dtype='object'),
 Index(['APPLICATION_NUMBER', 'FORM_NUMBER', 'FILE_DATE', 'STATUS_DATE',
        'STATUS', 'STATUS_CODE', 'EXPIRATION_DATE', 'ESTIMATED_COST',
        'REVISED_COST', 'EXISTING_USE', 'EXISTING_UNITS', 'PROPOSED_USE',
        'PROPOSED_UNITS', 'BLOCK', 'LOT', 'STREET_NUMBER', 'STREET_NUMBER_SFX',
        'AVS_STREET_NAME', 'AVS_STREET_SFX', 'UNIT', 'UNIT_SFX', 'FIRST_NAME',
        'LAST_NAME', 'CONTRACTORPHONE'

In [232]:
l = [1,2,3]
list(map({1:'hi'},l))

TypeError: 'dict' object is not callable

In [331]:
save(predictor)

In [206]:
predictor.columns

column_name_map = {}
for name in s:
    column_name_map[name] = name


In [239]:
column_name_map['proposedstories'] = 'proposed_stories'

In [241]:
with open('col_name_map.p', 'wb') as file:
    pickle.dump(column_name_map, file)

In [340]:
data[13].columns

Index(['application_number', 'form_number', 'file_date', 'status_date',
       'status', 'status_code', 'expiration_date', 'estimated_cost',
       'revised_cost', 'existing_use', 'existing_units', 'proposed_use',
       'proposed_units', 'block', 'lot', 'street_number', 'street_number_sfx',
       'avs_street_name', 'avs_street_sfx', 'unit', 'unit_sfx', 'first_name',
       'last_name', 'contractor_phone', 'company_name', 'street',
       'street_suffix', 'city', 'state', 'zip_code', 'contact_name',
       'contact_phone', 'description'],
      dtype='object')

In [348]:
old = data[0].columns
for i, datum in enumerate(data):
    if not compare(datum.columns,old, 50):
        print(i)
    old = datum.columns

13
72


In [4]:
predictor = pd.read_csv('data/predictor.csv', engine = 'python')
target = pd.read_csv('data/target.csv')

In [79]:
s = pd.Series([1,2,3])
s.astype(str)

0    1
1    2
2    3
dtype: object

In [89]:
predictor.columns;

In [94]:
#only issued applications matter
predictor = predictor[predictor['status'] == 'ISSUED'].drop('status', axis = 1)

SyntaxError: invalid syntax (<ipython-input-94-3e9599a24fc8>, line 2)

In [100]:
predictor['addr2'] = predictor['address'].apply(lambda x: x.split()).apply(lambda x: ' '.join(x[:-1]))

In [528]:
start

Timestamp('2004-04-01 00:00:00')

In [570]:
ind = 15

In [571]:
start = predictor['status_date'][ind]
address = predictor['address'][ind]
temp= target[target['date'] - start > pd.Timedelta('0 days')]

In [572]:
fires = temp[temp['Address'].apply(lambda x: address == x[:len(address)])]

In [584]:
predictor = predictor[predictor['status'] == 'ISSUED']

In [585]:
save(predictor)

In [ ]:
mask1 = (pd.Timedelta('1825 days') > (tdates - start) )
mask2 = (tdates - start) > pd.Timedelta('0 days')

In [604]:
save(predictor)

In [609]:
fires = np.zeros(len(predictor))

In [624]:
times = []
for i,time in enumerate(uniquetimes):
    try:
        times.append(pd.to_datetime(time))
    except:
        print(i, time)

In [ ]:
with open('data/')

In [672]:
data[74]['status_date']

0       2015-12-22 00:00:00
1       2015-12-22 00:00:00
2       2015-12-15 00:00:00
3       2015-12-09 00:00:00
4       2015-12-09 00:00:00
5       2015-12-09 00:00:00
6       2015-12-09 00:00:00
7       2015-12-30 00:00:00
8       2015-12-02 00:00:00
9       2015-12-11 00:00:00
10      2015-12-10 00:00:00
11      2015-12-10 00:00:00
12      2015-12-10 00:00:00
13      2015-12-10 00:00:00
14      2015-12-03 00:00:00
15      2015-12-22 00:00:00
16      2015-12-08 00:00:00
17      2015-12-01 00:00:00
18      2015-12-18 00:00:00
19      2015-12-11 00:00:00
20      2015-12-03 00:00:00
21      2015-12-29 00:00:00
22      2015-12-01 00:00:00
23      2015-12-01 00:00:00
24      2015-12-01 00:00:00
25      2015-12-01 00:00:00
26      2015-12-01 00:00:00
27      2015-12-07 00:00:00
28      2015-12-16 00:00:00
29      2015-12-29 00:00:00
               ...         
4952                  42369
4953                  42369
4954                  42369
4955                  42369
4956                

In [671]:
for i, x in enumerate(data):
    for j, row in x.iterrows():
        if type(row['status_date']) == int:
            print(i, j)
            break

74 4794
124 4794


In [444]:
[(i, type(x.iloc[5]['status_date'])) for (i,x) in enumerate(data)];

NameError: name 'data' is not defined

In [641]:
for i, (a, b) in enumerate(zip(times, list(times2))):
    if a != b:
        print(i)
        break


1578


In [659]:
[(i,type(x)) for (i,x) in enumerate(predictor['status_date'].unique())];

In [786]:
target['Address'];

In [781]:
for i, date in enumerate(predictor['status_date'].unique()):
    if i%500 == 0:
        print(i)
    try:
        fires_after[date] = target[target['date'] - date > pd.Timedelta('0 days')]

2554

In [433]:
pd.to_datetime(predictor['status_date']);

In [438]:
save(target, target=True)

In [455]:
target

,address,date
0,156 GEARY STREET,2019-03-16
1,BRYANT STREET,2019-03-15
2,1390 MISSION STREET,2019-03-15
3,421 TURK STREET,2019-03-15
4,1449 POTRERO AVENUE,2019-03-15
5,SOUTH VAN NESS AVENUE,2019-03-14
6,6TH STREET,2019-03-14
7,TOWNSEND STREET,2019-03-14
8,2887 CESAR CHAVEZ STREET,2019-03-14
15,30 MASON STREET,2019-03-13


In [166]:
fires = np.zeros((len(pred), 2))

In [170]:
fires[2,1] = 5
fires

array([[0., 0.],
       [0., 0.],
       [0., 5.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [179]:
y_df = target

In [257]:
predictor['street_number'] = predictor['street_number'].fillna(0).astype(int).astype(str)

In [ ]:
with open('data/predictor.p', 'rb') as file:
    predictor = pickle.load(file)

In [11]:
with open('data/fires_df.p', 'rb') as file:
    fires_df = pickle.load(file)

In [17]:
with open('target_date_index.p', 'rb') as file:
    target_date_index = pickle.load(file)

In [3]:
predictor = data_processing(predictor)

In [8]:
target_matching = predictor[['address', 'status_date']]

In [19]:
fires = np.zeros((len(target_matching), 2))

In [21]:
a, b, c = 0,0,0
start_date = pd.to_datetime('2004-01-01')

In [23]:
time1 = time.clock()
for i, row in target_matching.iterrows():
    if i%10000 == 0:
        print('done with {} iterations'.format(i))
        print(time.clock() - time1)
    try:
        #part a
#         p1 = time.clock()
        start = row['status_date']
        address = row['address']
        #part b
#         p2 = time.clock()
        
        days = (start - start_date).days
        after = fires_df.iloc[:target_date_index[days]]
        before = fires_df.iloc[target_date_index[days]:]
        
        #part c
#         p3 = time.clock()
        
        fires[i,0] = len(before[before['address'].apply(lambda x: address == x[:len(address)])])
        fires[i,1] = len(after[after['address'].apply(lambda x: address == x[:len(address)])])
#         last = time.clock()
        
#         a += p2 - p1
#         b += p3 - p2
#         c += last - p3
    except:h
        print(i)
        print(sys.exc_info())
        break

done with 0 iterations
1.064575700000006
done with 10000 iterations
420.06667889999994
done with 20000 iterations
999.9716087
done with 30000 iterations
1529.1829229999998
done with 40000 iterations
2049.0285359
done with 50000 iterations
2585.1689252
done with 60000 iterations
2955.7227112
done with 70000 iterations
3204.6128176
done with 80000 iterations
3472.4233127
done with 90000 iterations
3736.2976739
done with 100000 iterations
3990.6556754999997
done with 110000 iterations
4245.009769599999
done with 120000 iterations
4513.232611099999
done with 130000 iterations
4775.3978185999995
done with 140000 iterations
5030.5835447
done with 150000 iterations
5285.866743199999
done with 160000 iterations
5592.6793345
done with 170000 iterations
6108.840659199999
done with 180000 iterations
6602.8426583
done with 190000 iterations
7118.7964041
done with 200000 iterations
7802.007957
done with 210000 iterations
8494.1565317
done with 220000 iterations
9010.991085
done with 230000 iteratio

In [33]:
target = pd.DataFrame(data = fires, columns = ['fires_before', 'fires_after'])

In [37]:
target

,fires_before,fires_after,date
0,0.0,0.0,2004-04-01
1,0.0,0.0,2004-04-01
2,0.0,0.0,2004-04-01
3,0.0,0.0,2004-04-01
4,0.0,0.0,2004-04-01
5,1.0,1.0,2004-04-01
6,0.0,0.0,2004-04-01
7,0.0,0.0,2004-04-01
8,1.0,0.0,2004-04-01
9,0.0,0.0,2004-04-01


In [25]:
for i, row in predictor.iterrows():
    print(row['address'])
    break

705.0 SHOTWELL


In [34]:
a,b,c

(0, 0, 0)

In [487]:
end_date = target.iloc[0]['date']

In [493]:
time_period = (end_date - predictor['status_date']).apply(lambda x: x.days)

In [495]:
predictor['fire_rate'] = predictor['fires'] /time_period * 365.25

In [499]:
save(predictor)

In [484]:
target[target['address'].apply(lambda x: x[:14] == '330 CLEMENTINA')]

,address,date
228,330 CLEMENTINA STREET,2019-02-10
1013,330 CLEMENTINA ST 503,2018-11-10
3700,330 CLEMENTINA STREET,2018-02-09
4465,330 CLEMENTINA STREET,2017-12-03
6546,330 CLEMENTINA STREET,2017-05-04
6796,330 CLEMENTINA STREET,2017-04-05
6894,330 CLEMENTINA STREET,2017-03-23
7288,330 CLEMENTINA STREET,2017-01-31
8802,330 CLEMENTINA ST 9TH,2016-08-25
8936,330 CLEMENTINA ST 1011,2016-08-12


predictor

In [773]:
fires[0] = len(temp[temp['Address'].apply(lambda x: address == x[:len(address)])])

In [762]:
start = row['status_date']
address = row['address']

In [11]:
def int_able(string):
    try:
        x = int(string)
    except:
        return False
    return True

In [41]:
predictor[predictor['address'].apply(lambda x: test in x)]

,file_date,estimated cost,proposed use,proposed units,proposed stories,address,list


In [96]:
#get the rows there the first word of the address is a number
#in other words, make sure there an address number.
predictor['address'][predictor['address'].apply(lambda x: x.split()[0]).apply(intable)];

Confirmed: Every single row in this data has an address number that's the first word in the address


In [32]:
#let's get that list out
predictor['list'] = predictor['address'].apply(lambda x: x.split())

In [97]:
predictor['address'].apply(lambda x: x.split()[-1]).unique();

In [31]:
street_names = predictor['address'].apply(lambda x: x.split()[-2]).unique()

In [341]:
test = '156 GEARY'
target[target['address'].apply(lambda x: test in x)]

,address,date
0,156 GEARY STREET,2019-03-16


In [ ]:
#let's find stuff with the same second word but different suffix. see if that's a thing?
for name in street_names:
    predictor['address'].apply()

In [43]:
predictor['street name'] = predictor['list'].apply(lambda x: x[1])

In [45]:
predictor['addr2'] = predictor['list'].apply(lambda x: ' '.join(x[:-1]))

In [452]:
save(predictor)

In [278]:
predictor

,15_day_hold?,application_number,assessor_stories,avs_street_name,avs_street_sfx,block,city,company_name,contact_name,contact_phone,...,street,street_number,street_number.1,street_number_sfx,street_suffix,unit,unit_sfx,voluntary soft-story retrofit,zip_code,address
1,NaN,9.90103e+06,NaN,POWELL,ST,326,SAN FRANCISCO,C B F INC,NaN,NaN,...,HOWARD,111,NaN,NaN,ST,NaN,NaN,NaN,94105,111 POWELL
2,NaN,2.00404e+11,NaN,SHOTWELL,ST,3614,NaN,NaN,NaN,NaN,...,NaN,705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,705 SHOTWELL
3,NaN,2.00404e+11,NaN,26TH,ST,6564,SAN FRANCISCO,DERESZYNSKI-DUVAL & ASSOCIATES INC.,NaN,NaN,...,ILLINOIS,4089,NaN,NaN,ST,NaN,NaN,NaN,94124-0000,4089 26TH
4,NaN,2.00404e+11,NaN,SUTTER,ST,676,NaN,NaN,NaN,NaN,...,NaN,1830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1830 SUTTER
5,NaN,2.00404e+11,NaN,ESPANOLA,ST,4713,SAN FRANCISCO,MARINSHIP CONSTRUCTION,MIA CARTER,7162443.0,...,DORMAN,20,NaN,NaN,AV,NaN,NaN,NaN,94124,20 ESPANOLA
6,NaN,2.00404e+11,NaN,03RD,ST,3706,OAKLAND,ARROW SIGN CO,NaN,NaN,...,46TH,50,NaN,NaN,AV,NaN,NaN,NaN,94601,50 03RD
7,NaN,2.00404e+11,NaN,47TH,AV,2083,NaN,NaN,PAT BUSCOVICH 235 MONTGOMERY ST SF,NaN,...,NaN,1812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1812 47TH
8,NaN,2.00404e+11,NaN,03RD,ST,5242,STOCKTON,"MINA-TREE SIGNS, INC",NaN,NaN,...,E. RONALD,3801,NaN,NaN,ST,NaN,NaN,NaN,95208,3801 03RD
9,NaN,2.00404e+11,NaN,BALDWIN,CT,4714,SAN FRANCISCO,MARINSHIP CONSTRUCTION,NaN,NaN,...,DORMAN,51,NaN,NaN,AV,NaN,NaN,NaN,94124,51 BALDWIN
10,NaN,2.00404e+11,NaN,MARIETTA,DR,2954A,NaN,NaN,JERRY BROWN DESIGN,NaN,...,NaN,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190 MARIETTA


In [271]:
[name for name in dir() if name[0] != '_']

['BeautifulSoup',
 'In',
 'LinearRegression',
 'LogisticRegression',
 'Out',
 'X',
 'a',
 'address',
 'after',
 'b',
 'before',
 'c',
 'col',
 'column_name_map',
 'columns',
 'data',
 'data_pipeline',
 'date_cols',
 'days',
 'df',
 'exit',
 'file',
 'fires',
 'fires_df',
 'get_ipython',
 'i',
 'int_to_date',
 'last',
 'mar_2004_cols',
 'model',
 'np',
 'os',
 'p1',
 'p2',
 'p3',
 'path',
 'pd',
 'pickle',
 'pred',
 'predictor',
 'quit',
 're',
 'requests',
 'row',
 'save',
 'start',
 'start_date',
 'string',
 'sys',
 'target',
 'target_date_index',
 'target_pipeline',
 'test',
 'time',
 'y',
 'y_df']

NameError: name 'vec' is not defined